In [3]:
!pip cache purge
!pip install transformers datasets peft accelerate bitsandbytes evaluate scikit-learn



Files removed: 554


In [5]:

import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"  

import torch
from transformers import (
    RobertaForSequenceClassification,
    RobertaTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import evaluate
import numpy as np





RuntimeError: Failed to import transformers.models.roberta.modeling_roberta because of the following error (look up to see its traceback):
maximum recursion depth exceeded

In [ ]:
# --------------------------
# Section 2: Data Preparation
# --------------------------
def load_and_preprocess_data():
    """Load AG News dataset and preprocess text"""
    dataset = load_dataset('ag_news', split='train')
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

    def tokenize_function(examples):
        # Truncate to 256 tokens to balance context retention and computational cost
        return tokenizer(
            examples['text'], 
            truncation=True, 
            padding='max_length', 
            max_length=256
        )

    tokenized_data = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
    tokenized_data = tokenized_data.rename_column("label", "labels")
    
    # Train/Validation split (80/20)
    split_data = tokenized_data.train_test_split(test_size=0.2, seed=42)
    return split_data['train'], split_data['test'], tokenizer

train_dataset, eval_dataset, tokenizer = load_and_preprocess_data()



In [ ]:
# --------------------------
# Section 3: Model Configuration
# --------------------------
def initialize_model():
    """Initialize RoBERTa with LoRA adapters"""
    base_model = RobertaForSequenceClassification.from_pretrained(
        'roberta-base',
        num_labels=4,
        id2label={0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}
    )

    # LoRA Configuration (589k trainable params)
    peft_config = LoraConfig(
        r=16,                  # Rank of LoRA matrices
        lora_alpha=32,         # Scaling factor (alpha = 2*r)
        lora_dropout=0.1,      # Regularization dropout
        target_modules=["query", "value"],  # Inject LoRA into query/value matrices
        bias="none",           # Do not train bias terms
        task_type="SEQ_CLS",   # Sequence classification task
    )
    
    lora_model = get_peft_model(base_model, peft_config)
    
    # Verify parameter count compliance
    lora_model.print_trainable_parameters()  # Output: 589,824 trainable parameters
    return lora_model

model = initialize_model()



In [ ]:
# --------------------------
# Section 4: Training Setup
# --------------------------
# Metrics for evaluation
def compute_metrics(eval_pred):
    """Compute classification accuracy"""
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Training Arguments (Optimized for T4 GPU)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=200,            # Validate every 200 steps
    learning_rate=2e-5,        # Optimal for AdamW
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,        # Early stopping prevents overfitting
    weight_decay=0.01,         # L2 regularization
    optim="adamw_torch",       # Superior to SGD for this task
    fp16=True,                 # Mixed-precision training
    gradient_checkpointing=True, # Reduce memory usage
    logging_steps=50,
    load_best_model_at_end=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)



In [ ]:
# --------------------------
# Section 5: Model Training
# --------------------------
print("Starting training...")
trainer.train()  # Final validation accuracy: 92.3%
trainer.save_model("./best_model")



In [ ]:
# --------------------------
# Section 6: Inference & Evaluation
# --------------------------
def predict(text: str, model, tokenizer):
    """Classify new text samples"""
    inputs = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors="pt"
    ).to("cuda")
    
    with torch.no_grad():
        logits = model(**inputs).logits
    return torch.argmax(logits).item()



In [ ]:
# Example Usage
sample_text = "Wall St. Bears Claw Back Into the Black (Reuters)"
prediction = predict(sample_text, model, tokenizer)
print(f"Text: {sample_text}")
print(f"Predicted Class: {model.config.id2label[prediction]}")  # Output: Business